In [1]:
import numpy as np
import pandas as pd
import sys
from datetime import datetime
import skmap_bindings
from pathlib import Path
import time
import os
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'

gdal_opts = {
 'GDAL_HTTP_VERSION': '1.0',
 'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.tif',
}
n_threads = 96
bands_list = [1,]
x_off = 0
y_off = 0
x_size, y_size = (4004, 4004)
nodata_val = 255
compression_command = f"gdal_translate -a_nodata {nodata_val} -co COMPRESS=deflate -co TILED=TRUE -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 -co NUM_THREADS=8"
tmp_write_dir = '../gaps_comulative'

with open('../global_tiles_s1.csv') as file:
    tiles = [f'{line.rstrip()}' for line in file]
n_tiles = len(tiles)



In [2]:
print("####################################################################")

layer = 'fraction_of_gaps_from_1997_to_2022_in_agg'
# for i in range(0, n_tiles, n_threads):
# for i in range(7800, 16000, n_threads):
for i in range(15000, n_tiles, n_threads):
    tiles_chunk = tiles[i:min(i+n_threads, n_tiles)]
    n_chunk = len(tiles_chunk)
    start = time.time()
    files_mask = [f'../../GPW/global_landmask/gpw_landmask_{tile}_ultimate.tif' for tile in tiles_chunk]
    files_out = [f'{layer}_{tile}_landmasked' for tile in tiles_chunk]
    files_in = [f'http://192.168.49.30:8333/prod-landsat-ard2/{tile}/{layer}.tif' for tile in tiles_chunk]
    mask_data = np.empty((n_chunk, x_size * y_size), dtype=np.float32)
    mask_idx = range(n_chunk)
    in_idx = range(n_chunk)
    in_data = np.empty((n_chunk, x_size * y_size), dtype=np.float32)
    
    skmap_bindings.readData(mask_data, n_threads, files_mask, mask_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)
    skmap_bindings.readData(in_data, n_threads, files_in, in_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)

    skmap_bindings.maskData(in_data, n_threads, in_idx, mask_data, 0., 255.)

    skmap_bindings.writeByteData(in_data, n_threads, gdal_opts, files_in, tmp_write_dir, files_out, in_idx,
        x_off, y_off, x_size, y_size, nodata_val, compression_command)
    # if ((i % (96 * 50)) == 0):
    print(f"Processing chunk from {i} to {min(i+n_threads, n_tiles)}", flush=True)
    print(f"Layer {layer}")
    print(f"Processed files in {time.time() - start:.2f} s", flush=True)
    
    
# print("####################################################################")
    
# layer = 'fraction_of_gaps_from_1997_to_2022_in_seasconv'
# for i in range(0, n_tiles, n_threads):
#     tiles_chunk = tiles[i:min(i+n_threads, n_tiles)]
#     n_chunk = len(tiles_chunk)
#     start = time.time()
#     files_mask = [f'../../GPW/global_landmask/gpw_landmask_{tile}_ultimate.tif' for tile in tiles_chunk]
#     files_out = [f'{layer}_{tile}_landmasked' for tile in tiles_chunk]
#     files_in = [f'http://192.168.49.30:8333/prod-landsat-ard2/{tile}/{layer}.tif' for tile in tiles_chunk]
#     mask_data = np.empty((n_chunk, x_size * y_size), dtype=np.float32)
#     mask_idx = range(n_chunk)
#     in_idx = range(n_chunk)
#     in_data = np.empty((n_chunk, x_size * y_size), dtype=np.float32)
    
#     skmap_bindings.readData(mask_data, n_threads, files_mask, mask_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)
#     skmap_bindings.readData(in_data, n_threads, files_in, in_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)

#     skmap_bindings.maskData(in_data, n_threads, in_idx, mask_data, 0., 255.)

#     skmap_bindings.writeByteData(in_data, n_threads, gdal_opts, files_in, tmp_write_dir, files_out, in_idx,
#         x_off, y_off, x_size, y_size, nodata_val, compression_command)
#     if ((i % (96 * 50)) == 0):
#         print(f"Processing chunk from {i} to {min(i+n_threads, n_tiles)}", flush=True)
#         print(f"Layer {layer}")
#         print(f"Processed files in {time.time() - start:.2f} s", flush=True)
    
    
# print("####################################################################")

    
# layer = 'fraction_of_gaps_from_2000_to_2022_in_agg'
# for i in range(0, n_tiles, n_threads):
#     tiles_chunk = tiles[i:min(i+n_threads, n_tiles)]
#     n_chunk = len(tiles_chunk)
#     start = time.time()
#     files_mask = [f'../../GPW/global_landmask/gpw_landmask_{tile}_ultimate.tif' for tile in tiles_chunk]
#     files_out = [f'{layer}_{tile}_landmasked' for tile in tiles_chunk]
#     files_in = [f'http://192.168.49.30:8333/prod-landsat-ard2/{tile}/{layer}.tif' for tile in tiles_chunk]
#     mask_data = np.empty((n_chunk, x_size * y_size), dtype=np.float32)
#     mask_idx = range(n_chunk)
#     in_idx = range(n_chunk)
#     in_data = np.empty((n_chunk, x_size * y_size), dtype=np.float32)
    
#     skmap_bindings.readData(mask_data, n_threads, files_mask, mask_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)
#     skmap_bindings.readData(in_data, n_threads, files_in, in_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)

#     in_data = ((in_data * 6. * 26. / 250.) - 6. * 26. + 6. * 23) / 6. / 23. * 250.
#     skmap_bindings.maskData(in_data, n_threads, in_idx, mask_data, 0., 255.)

#     skmap_bindings.writeByteData(in_data, n_threads, gdal_opts, files_in, tmp_write_dir, files_out, in_idx,
#         x_off, y_off, x_size, y_size, nodata_val, compression_command)
#     if ((i % (96 * 50)) == 0):
#         print(f"Processing chunk from {i} to {min(i+n_threads, n_tiles)}", flush=True)
#         print(f"Layer {layer}")
#         print(f"Processed files in {time.time() - start:.2f} s", flush=True)
# print("####################################################################")
    

    
# layer = 'fraction_of_gaps_from_2000_to_2022_in_seasconv'
# for i in range(0, n_tiles, n_threads):
#     tiles_chunk = tiles[i:min(i+n_threads, n_tiles)]
#     n_chunk = len(tiles_chunk)
#     start = time.time()
#     files_mask = [f'../../GPW/global_landmask/gpw_landmask_{tile}_ultimate.tif' for tile in tiles_chunk]
#     files_out = [f'{layer}_{tile}_landmasked' for tile in tiles_chunk]
#     files_in = [f'http://192.168.49.30:8333/prod-landsat-ard2/{tile}/{layer}.tif' for tile in tiles_chunk]
#     mask_data = np.empty((n_chunk, x_size * y_size), dtype=np.float32)
#     mask_idx = range(n_chunk)
#     in_idx = range(n_chunk)
#     in_data = np.empty((n_chunk, x_size * y_size), dtype=np.float32)
    
#     skmap_bindings.readData(mask_data, n_threads, files_mask, mask_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)
#     skmap_bindings.readData(in_data, n_threads, files_in, in_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)

#     in_data = in_data * 26. / 23. 
#     skmap_bindings.maskData(in_data, n_threads, in_idx, mask_data, 0., 255.)

#     skmap_bindings.writeByteData(in_data, n_threads, gdal_opts, files_in, tmp_write_dir, files_out, in_idx,
#         x_off, y_off, x_size, y_size, nodata_val, compression_command)
#     if ((i % (96 * 50)) == 0):
#         print(f"Processing chunk from {i} to {min(i+n_threads, n_tiles)}", flush=True)
#         print(f"Layer {layer}") 
#         print(f"Processed files in {time.time() - start:.2f} s", flush=True)
# print("####################################################################")


####################################################################
Processing chunk from 15000 to 15096
Layer fraction_of_gaps_from_1997_to_2022_in_agg
Processed files in 4.17 s
Processing chunk from 15096 to 15192
Layer fraction_of_gaps_from_1997_to_2022_in_agg
Processed files in 2.52 s
Processing chunk from 15192 to 15288
Layer fraction_of_gaps_from_1997_to_2022_in_agg
Processed files in 2.76 s
Processing chunk from 15288 to 15384
Layer fraction_of_gaps_from_1997_to_2022_in_agg
Processed files in 5.78 s
Processing chunk from 15384 to 15480
Layer fraction_of_gaps_from_1997_to_2022_in_agg
Processed files in 2.59 s
Processing chunk from 15480 to 15576
Layer fraction_of_gaps_from_1997_to_2022_in_agg
Processed files in 2.53 s
Processing chunk from 15576 to 15672
Layer fraction_of_gaps_from_1997_to_2022_in_agg
Processed files in 2.83 s
Processing chunk from 15672 to 15768
Layer fraction_of_gaps_from_1997_to_2022_in_agg
Processed files in 2.63 s
Processing chunk from 15768 to 15864
Layer 